In [1]:
!pip install pandas langchain langchain-community sentence-transformers faiss-cpu "transformers[agents]" --upgrade -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-llms-huggingface 0.4.1 requires huggingface-hub<0.24.0,>=0.23.0, but you have huggingface-hub 0.27.0 which is incompatible.
pr-agent 0.2.4 requires openai==1.55.3, but you have openai 1.57.4 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install ipywidgets

   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 2.3/2.3 MB 22.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
import datasets

knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

c:\Users\AjaykumarKV\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\download\streaming_download_manager.py:765: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


In [7]:
knowledge_base

Dataset({
    features: ['text', 'source'],
    num_rows: 2647
})